### 参考文献结构信息格式化

根据 GB/T 7714-2015 信息与文献 参考文献著录规则 中的要求实现，可能存在一些格式标准问题，可以根据具体要求进行更改。

一个简单的将结构信息格式化的方法，主要处理了空缺信息下的格式成分变化的问题。

In [176]:
import re
from collections import UserDict


class DefaultDict(UserDict):
    def __missing__(self, key):
        return f"{{{key}}}"


def wrap(fmt, data, return_str=""):
    try:
        res = fmt.format_map(data)
    except:
        res = fmt if return_str is None else return_str
    return res


class ReplaceFunc:
    @property
    def flag(self):
        flag = self._flag
        self._flag = False
        return flag

    def __init__(self, data):
        self.data = data
        self._flag = True

    def __call__(self, match):
        self._flag = True
        fmt = match.groups()[0]
        fmt = fmt.replace("&lt;", "<").replace("&gt;", ">")
        return wrap(fmt, self.data)


def wrapFormat(fmt, data):
    fmt = fmt.replace("\<", "&lt;").replace("\>", "&gt;")
    data = {k: v for k, v in data.items() if v}
    fun = ReplaceFunc(data)

    while fun.flag:
        fmt = re.sub("<([^<>]*)>", fun, fmt)

    fmt = wrap(fmt, DefaultDict(data), None)
    fmt = fmt.replace("&lt;", "<").replace("&gt;", ">")
    return fmt


In [7]:
from sympy import expand
import yaml


def parseYAML(file):
    with open(file, encoding="utf-8") as f:
        data = yaml.load(f, yaml.FullLoader)
    return data


examples = parseYAML("./examples.yaml")
formats = parseYAML("./formats.yaml")


# wrapFormat(formats["专著"], examples["structed"][0])


import json

# print(json.dumps(formats, ensure_ascii=False))
print(json.dumps(examples, ensure_ascii=False))
# json.load()


{"structed": [{"类型": "专著", "主要责任者": "陈登原", "题名": "国史旧闻", "其他题名信息": "第1卷", "文献类型标识": "M", "文献载体标识": null, "其他责任者": null, "版本项": null, "出版地": "北京", "出版者": "中华书局", "出版年": 2000, "引文页码": 29, "引用日期": null, "获取和访问路径": null, "数字对象唯一标识符": null}, {"类型": "专著", "主要责任者": "哈里森,沃尔德伦", "题名": "经济数学与金融数学", "其他题名信息": null, "文献类型标识": "M", "文献载体标识": null, "其他责任者": "谢远涛,译", "版本项": null, "出版地": "北京", "出版者": "中国人民大学出版社", "出版年": 2012, "引文页码": "235-236", "引用日期": null, "获取和访问路径": null, "数字对象唯一标识符": null}, {"类型": "专著", "主要责任者": "北京市政协民族和宗教委员会,北京联合大学民族与宗教研究所", "题名": "历代王朝与民族宗教", "其他题名信息": null, "文献类型标识": "M", "文献载体标识": null, "其他责任者": null, "版本项": null, "出版地": "北京", "出版者": "民族出版社", "出版年": 2012, "引文页码": 112, "引用日期": null, "获取和访问路径": null, "数字对象唯一标识符": null}, {"类型": "专著", "主要责任者": "全国信息与文献标准化技术委员会", "题名": "信息与文献 都柏林核心元数据元素集", "其他题名信息": "GB/T 25100—2010", "文献类型标识": "S", "文献载体标识": null, "其他责任者": null, "版本项": null, "出版地": "北京", "出版者": "中国标准出版社", "出版年": 2010, "引文页码": "2-3", "引用日期": null, "获取和访问路径": null, "数字对象唯一标识符": nul

### 杂项

In [ ]:
# 标点格式
marks = {
    "．": ". ",
    "，": ", ",
    "［": "[",
    "］": "]",
    "—": "-",
}
# http://wjk.usst.edu.cn/

### Old Version: 值和格式的二元组

In [ ]:
# (content, fmt) -> fmt.format(content) if fmt else ""

def wrap(content: str, wrapper: str):
    return wrapper.format(content) if content else ""


def wrap_with_fmt(fmt: list, data=None):
    if len(fmt) == 2:
        return wrap(data.get(fmt[0], "") if data else fmt[0], fmt[1])

    s = "".join([wrap_with_fmt(fmt_i, data) for fmt_i in fmt[:-1]])

    return wrap(s, fmt[-1])


In [ ]:
# format_map 可以接受字典，直接使用整个字典表示content

p = ("{a}", ["[", "{}", "/{b}", "]"])


def wrapfmt(s, data):
    if isinstance(s, tuple):
        return wrap(wrapfmt(s[0], data), wrapfmt(s[1], data))
    elif isinstance(s, str):
        try:
            res = s.format("{}", **data)
        except:
            res = ""
        return res
    elif isinstance(s, list):
        return "".join([wrapfmt(i, data) for i in s])


# wrapfmt(p, {"a": "1", "c": "23"})


### Test: 正则表达式分析

In [190]:
import re

regex = "(\[\d+\])\s+(?:(?!http)(.+?)\.)?(?:(?!http)(.+?)\.)?(?:(?!http)(.+?)\.)?(?:(?!http)(.+?)\.)?(?:(?:(?:(https?.*)\.(?=DOI))?(DOI.*)\.)|(?:(https?.*)\.$))"

a = re.match(regex, examples["origin"][-2])

a.groups()



KeyError: 'origin'